In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
# import os
# db_username = os.getenv('DB_USERNAME')
# db_password = os.getenv('DB_PASSWORD')

In [4]:
headers = {'user-agent':'Mozilla/5.0 \
            (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/84.0.4147.105 Safari/537.36'}

In [5]:
symbols = [
   'AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'TSLA', 'AVGO', 'BRK-B', 'GOOG' 
]

In [6]:
all=[]
for symbol in symbols:
    url = 'https://finance.yahoo.com/quote/'+symbol+'/'
    page = requests.get(url,headers=headers)

    profile_url = 'https://finance.yahoo.com/quote/'+symbol+'/profile/'
    description_page = requests.get(profile_url,headers=headers)

    try:
        soup = BeautifulSoup(page.text, 'html.parser')
        
        company_full_name = soup.find('h1', {'class': 'yf-xxbei9'}).text.split('(')
        company_name = company_full_name[0].strip()
        company_symbol = company_full_name[1].replace(')', '').strip()

        price = soup.find('span', {'data-testid': 'qsp-price'}).text
        
        change = soup.find('span', {'data-testid': 'qsp-price-change'}).text  

        fin_streamer = soup.find('fin-streamer', attrs={'data-symbol': {company_symbol}, 'data-field': 'regularMarketVolume'})
        volume = fin_streamer.text.strip()

        #description
        soup = BeautifulSoup(description_page.text, 'html.parser')
        description = soup.find('section', {'data-testid': 'description'}).find('p').text
        description = description[:250]

        x=[company_name,company_symbol,price,change,volume,url, description]
        all.append(x)

    except AttributeError:
        print("Change the Element id")
        # Wait for a short time to avoid rate limiting
        time.sleep(10)

In [7]:
#panda it
column_names = ["Company", "Symbol","Price", "Change","Volume","url"]
df = pd.DataFrame(columns = column_names)
for i in all:
  index=0
  df.loc[index] = i
  df.index = df.index + 1
df=df.reset_index(drop=True)
df.to_csv('stocks.CSV')

ValueError: cannot set a row with mismatched columns